In [4]:
import webbrowser
import requests
import msal
import json

import os
os.chdir("/home/daniel/de_AI_Fabriek/belasting-agent-playground/")

from keys import O365_CLIENT_SECRET, O365_CLIENT_ID

# SCOPES = ["User.Read"]
SCOPES = ['Calendars.ReadWrite.Shared']
GRAPH_API_ENDPOINT = 'https://graph.microsoft.com/v1.0'

# AUTHORITY = 'https://login.microsoftonline.com/' + "b4790366-ebc1-458b-8baf-4aed4d12f0fc"
AUTHORITY = 'https://login.microsoftonline.com/common'

In [22]:
import webbrowser
from datetime import datetime
import json
import os
import msal


def generate_access_token(app_id, scopes):
    # Save Session Token as a token file
    access_token_cache = msal.SerializableTokenCache()

    # read the token file
    if os.path.exists('msg_api_token_access.json'):
        access_token_cache.deserialize(open("msg_api_token_access.json", "r").read())
        token_detail = json.load(open('msg_api_token_access.json',))
        token_detail_key = list(token_detail['AccessToken'].keys())[0]
        token_expiration = datetime.fromtimestamp(int(token_detail['AccessToken'][token_detail_key]['expires_on']))
        if datetime.now() > token_expiration:
            os.remove('msg_api_token_access.json')
            access_token_cache = msal.SerializableTokenCache()

    # assign a SerializableTokenCache object to the client instance
    client = msal.PublicClientApplication(client_id=app_id, token_cache=access_token_cache)

    accounts = client.get_accounts()
    if accounts:
        # load the session
        token_response = client.acquire_token_silent(scopes, accounts[0])
    else:
        # authetnicate your accoutn as usual
        flow = client.initiate_device_flow(scopes=scopes)
        print('user_code: ' + flow['user_code'])
        webbrowser.open('https://microsoft.com/devicelogin')
        token_response = client.acquire_token_by_device_flow(flow)

    with open('msg_api_token_access.json', 'w') as _f:
        _f.write(access_token_cache.serialize())

    return token_response




In [23]:
access_token = generate_access_token(O365_CLIENT_ID, SCOPES)
headers = {
    'Authorization': 'Bearer ' + access_token['access_token']
}

# create an event


In [7]:
def construct_event_detail(event_name, **kwargs):
    request_body = {
        'subject': event_name,
    }
    for key, value in kwargs.items():
        request_body[key] = value
    return request_body


In [12]:
response1_create = requests.post(
    GRAPH_API_ENDPOINT + f'/me/events',
    headers=headers,
    json=construct_event_detail("test_event2")
)

print(json.dumps(response1_create.json(), indent=4))

{
    "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users('outlook_4C0C63D7D80EDD22%40outlook.com')/events/$entity",
    "@odata.etag": "W/\"lr+kWymW20KA+/WCcZNXrQAHzHEZsQ==\"",
    "id": "AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD_k_4msGd5-CKLAcAlr_kWymW20KA_-WCcZNXrQAAAgENAAAAlr_kWymW20KA_-WCcZNXrQAHzC34jAAAAA==",
    "createdDateTime": "2025-02-05T10:54:10.6741639Z",
    "lastModifiedDateTime": "2025-02-05T10:54:10.7234921Z",
    "changeKey": "lr+kWymW20KA+/WCcZNXrQAHzHEZsQ==",
    "categories": [],
    "transactionId": null,
    "originalStartTimeZone": "UTC",
    "originalEndTimeZone": "UTC",
    "iCalUId": "040000008200E00074C5B7101A82E00800000000EF856949BC77DB010000000000000000100000003B22806BE297104BBCD6EBA787C7B778",
    "uid": "040000008200E00074C5B7101A82E00800000000EF856949BC77DB010000000000000000100000003B22806BE297104BBCD6EBA787C7B778",
    "reminderMinutesBeforeStart": 15,
    "isReminderOn": true,
    "hasAttachments": false,
    "subject":

In [13]:
print(json.dumps(response1_create.json(), indent=2))


{
  "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users('outlook_4C0C63D7D80EDD22%40outlook.com')/events/$entity",
  "@odata.etag": "W/\"lr+kWymW20KA+/WCcZNXrQAHzHEZsQ==\"",
  "id": "AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD_k_4msGd5-CKLAcAlr_kWymW20KA_-WCcZNXrQAAAgENAAAAlr_kWymW20KA_-WCcZNXrQAHzC34jAAAAA==",
  "createdDateTime": "2025-02-05T10:54:10.6741639Z",
  "lastModifiedDateTime": "2025-02-05T10:54:10.7234921Z",
  "changeKey": "lr+kWymW20KA+/WCcZNXrQAHzHEZsQ==",
  "categories": [],
  "transactionId": null,
  "originalStartTimeZone": "UTC",
  "originalEndTimeZone": "UTC",
  "iCalUId": "040000008200E00074C5B7101A82E00800000000EF856949BC77DB010000000000000000100000003B22806BE297104BBCD6EBA787C7B778",
  "uid": "040000008200E00074C5B7101A82E00800000000EF856949BC77DB010000000000000000100000003B22806BE297104BBCD6EBA787C7B778",
  "reminderMinutesBeforeStart": 15,
  "isReminderOn": true,
  "hasAttachments": false,
  "subject": "test_event2",
  "bodyPreview":

In [14]:
event_id1 = response1_create.json()['id']
response1_delete = requests.delete(
    GRAPH_API_ENDPOINT + f'/me/events/{event_id1}',
    headers=headers
)

## msal

In [18]:
def aquire_and_use_token(app_id, scopes, authority):
    access_token_cache = msal.SerializableTokenCache()
    
    # read token cache
    if os.path.exists("msg_api_token_access.json"):
        access_token_cache.deserialize(open("msg_api_token_access.json", "r").read())
        token_detail = json.load(open('msg_api_token_access.json',))
        token_detail_key = list(token_detail['AccessToken'].keys())[0]
        token_expiration = datetime.fromtimestamp(int(token_detail['AccessToken'][token_detail_key]['expires_on']))
        if datetime.now() > token_expiration:
            os.remove('msg_api_token_access.json')
            access_token_cache = msal.SerializableTokenCache()

        
    client = msal.PublicClientApplication(
        app_id,
        authority=authority,
        token_cache=access_token_cache
    )
    
    token_response = None
     
    accounts = client.get_accounts()
                  
    if accounts:
        
        # Assuming the end user chose this one to proceed
        chosen_account = accounts[0]    
        
        # Now let's try to find a token in cache for this account
        # or by using refresh token
        token_response = client.acquire_token_silent(scopes, account=chosen_account)

    if not token_response:
        flow = client.initiate_device_flow(scopes=scopes)
        print('user code: ', flow['user_code'])
        webbrowser.open(flow['verification_uri'])
        
        token_response = client.acquire_token_by_device_flow(flow)
        print(flow["message"])
    
    if "access_token" in token_response:
        print("Token was obtained from:", token_response["token_source"])
        if os.getenv('ENDPOINT'):
            # Calling a web API using the access token
            api_result = requests.get(
                os.getenv('ENDPOINT'),
                hearders={'Authorization': 'Bearer ' + token_response["access_token"]},
            ).json()    
            print("Web API call result", json.dumps(api_result, indent=2))
        else:
            print("Token acquisition result", json.dumps(token_response, indent=2))
    else:
        print("Token acquisition failed", token_response)
    
    # write to persistent cache
    with open("msg_api_token_access.json", "w") as _f:
        _f.write(access_token_cache.serialize())
    return token_response
    

In [21]:
token_response = aquire_and_use_token(O365_CLIENT_ID, SCOPES, AUTHORITY)

Token was obtained from: cache
Token acquisition result {
  "access_token": "EwB4A8l6BAAUBKgm8k1UswUNwklmy2v7U/S+1fEAASr1bpqc+uof1OfcwkymOC9xaGuN4GhkTyiLnWrMt/37VP3ylxwOAef+845oQSTnN6Zln/jMSRqfTubZj5/cP0kYGcV973ISpXMEUvwwN8VRQ3sCZZ1j6uBr2LZnsC1dh9na0XdqClV10VY+lMpzy2/Yh1bCGW4Abm7vT3Koq+NNO6H5MenxXP7trIYfbOQ5YhavkbM6n10awqJDHT9iiIe1QX4CBulQEydZwGx+2bgjrtm1urgYqRfmLqud3JiVWKdF6n8+H3IhwYaClkmK6Kpn3L0pTav+1fUHeWJ/DS0zJBy4Qp+/SQntyW6c9YppgBZIUpeM/DqiRLdcd1+tEqEQZgAAENKHonKeX3v/+oSqravd9cZAAp4+8u0A9CjoznrVf87RuWM9/BKhFkGF1e+Qw5NdOh5GWHa3XGXCFboS92dHsJiWZgWXDsT3/p0SXSxMr3UzI+N3EQssAR0eUacN7WP0GRnqjPDT0D4Le+GFpkQ8bFAZ2ndPXFDcaUMXCtM8miJbzWQf1cvZd9BS0u6yeTzFvchel8ZmZtWa8kiP2H+K6ngd6nfP99sAjje2UbSATZeBKEUwqXmuoDba2La7tsD6fqyqGYOm5mvImPEW2QermNVh/gRhmXxMBO2ANSC7OlUDLtN5UxK+NuzELyVH/DbQhZtV1mpM53eGYDLrWj9ldjhNxpeZZAuX1MhKUE9kBZKmwjGEIhQWSOP57BhitaLNYFAhW6l3souRfUu+2GfI1uOIBk4VnuwDzqRH6Dc7psTgB+Ompv8QzhZ+iTpGVBIVpo3RcIt3y8mMnTyFL0xO6PiA3C/82s4tf4064RTj3VrqbrlwiyNNa1fAmcDkwN7G3slB+gErQk/Io+lCczLo2Mr

In [5]:
token_response


{'token_type': 'Bearer',
 'scope': 'openid profile User.Read email',
 'expires_in': 4362,
 'ext_expires_in': 4362,
 'access_token': 'eyJ0eXAiOiJKV1QiLCJub25jZSI6ImlSR2ViNUdrV2FUS0wzbXJIM1hXSXl4clJJaHV2R3pfRkE3dWRNUXdzRzAiLCJhbGciOiJSUzI1NiIsIng1dCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyIsImtpZCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyJ9.eyJhdWQiOiIwMDAwMDAwMy0wMDAwLTAwMDAtYzAwMC0wMDAwMDAwMDAwMDAiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8wMDMwZTJmOC05Njg4LTQ5YzgtYWU0Zi1jZWY2N2E3ZWM4ZmYvIiwiaWF0IjoxNzM4NzA4NTY4LCJuYmYiOjE3Mzg3MDg1NjgsImV4cCI6MTczODcxMzIzMSwiYWNjdCI6MCwiYWNyIjoiMSIsImFpbyI6IkFUUUF5LzhaQUFBQXVvMGVPQlhCVUtYSDh1WEhQak9hMDRVK2RhZm9yQ3JyS2hEYllzbTg3K3hrNmN3Wmc3bEVJcElmekdQbWc3dmUiLCJhbXIiOlsicHdkIl0sImFwcF9kaXNwbGF5bmFtZSI6ImJlbGFzdGluZy1hYW5naWZ0ZW4tYWdlbnQiLCJhcHBpZCI6IjZmZTRjOTlhLTc4ODYtNDQ2My05MWJiLWNmMTMzYzVkZDE4NCIsImFwcGlkYWNyIjoiMCIsImZhbWlseV9uYW1lIjoidmFuIE9vc3Rlcm9vbSIsImdpdmVuX25hbWUiOiJEYW5pZWwiLCJpZHR5cCI6InVzZXIiLCJpcGFkZHIiOiIyMDAxOjFjMDA6ZDA0OjIyMDA6ZmUyMDo4ND